In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import Polynomial
from math import factorial
from scipy.linalg import solveh_banded
np.set_printoptions(precision=3, linewidth=200)
from scipy.special import roots_legendre, roots_chebyt
from scipy.integrate import quad
from scipy.misc import derivative
from scipy.stats import qmc

In [2]:
def get_points(a, b, n, mode):
    if mode == 'uniform':
        points = np.linspace(a, b, n)
    elif mode == 'random':
        points = np.concatenate(([a], np.sort(np.random.rand(n - 2) * (b - a) + a), [b]))
    elif mode == 'chebyshev':
        points = (np.polynomial.chebyshev.chebpts1(n) + 1) / 2 * (b - a) + a
    else:
        print(f'incorrect mode {mode}')
        points = 0
    return np.array(points)

In [3]:
def gauss_legendre(n, f, verbose=False):
    a = 0
    b = np.pi/2
    points = roots_legendre(n)[0]
    points_projected = (points + 1) / 2 * (b - a) + a
    l = Polynomial.fromroots(points)
    w = l.deriv()
    d = np.zeros(n)
    for i, x in enumerate(points):
        l_i  = Polynomial.fromroots(list(points[:i]) + list(points[i+1:])) / w(x)
        l_i_integr = l_i.integ()
        d[i] = l_i_integr(1) - l_i_integr(-1)
        d[i] *= (b - a) / 2
    if verbose:    
        print('d', d)
    fx = f(points_projected)
    res = np.sum(fx * d)
    print('quadrature result =', res)
    integration = quad(f, a, b)
    print('gauss-legendre error =', np.abs(res - integration[0]), '\n')

In [4]:
def proj(a, b, x):
    # x from [a, b] to [-1,1]
    return 2 * (x - a) / (b - a) - 1

def proj_inv(a, b, x):
    # x from [-1, 1] to [a, b]
    return (x * (b - a) + a + b) / 2

def clenshaw_curtis(n, f, verbose=False):
    a = 0
    b = np.pi/2
    points = np.array([np.cos(np.pi * i / n) for i in range(n)])
    halfn = n // 2
    coef = 2 / (b - a)
    numer = np.zeros(halfn)
    denom = np.array([1] + [2 / (1 - 4*k**2) for k in range(1, halfn)])
    for k in range(halfn):
        a2k = (f(proj_inv(a, b, 1)) + f(proj_inv(a, b, -1))) / 2
        for j in range(1, n):
            a2k += f(proj_inv(a, b, np.cos(np.pi * j / n))) * np.cos(j * 2 * k * np.pi / n)
        numer[k] = 2 * a2k / n
    if verbose:
        print(coef)
        
        print(numer)
        print(denom)
    res = np.sum(numer * denom) / coef
    print('quadrature result =', res)
    integration = quad(f, a, b)
    print('clenshaw-curtis error =', np.abs(res - integration[0]), '\n')
    return np.abs(res - integration[0])

In [5]:
def agm(a, b, eps, verbose=False):
    a_prev = a
    b_prev = b
    a_next = np.sqrt(a * b) 
    b_next = (a + b) / 2
    n = 1
    while np.abs(b_next - a_next) >= eps:
        a_next = np.sqrt(a_prev * b_prev)
        b_next = (a_prev + b_prev) / 2
        a_prev = a_next
        b_prev = b_next
        n += 1
    res = np.pi / 2 / a_next
    print(f'{n} agm res =', res)
    integration = quad(f, 0, np.pi/2)
    print('error =', np.abs(res - integration[0]))

In [6]:
def get_f(a, b):
    def f(x):
        return 1 / np.sqrt(np.power(a * np.sin(x), 2) + np.power(b * np.cos(x), 2))
    return f

In [55]:
a = 5
b = 10
f = get_f(a, b)
integration = quad(f, 0, np.pi/2)
print('scipy integration result =', integration[0], '\n')
gauss_legendre(15, f)
eps = clenshaw_curtis(15, f)
agm(a, b, eps)
agm(a, b, 1e-10)

scipy integration result = 0.21565156474996452 

quadrature result = 0.2156515647500356
gauss-legendre error = 7.108202915162565e-14 

quadrature result = 0.21565156417609468
clenshaw-curtis error = 5.738698405366449e-10 

5 agm res = 0.2156515647499643
error = 2.220446049250313e-16
5 agm res = 0.2156515647499643
error = 2.220446049250313e-16


In [56]:
a = 1234
b = 5321
f = get_f(a, b)
integration = quad(f, 0, np.pi/2)
print('scipy integration result =', integration[0], '\n')
gauss_legendre(15, f)
eps = clenshaw_curtis(15, f)
agm(a, b, eps)
agm(a, b, 1e-10)

scipy integration result = 0.0005399826375217206 

quadrature result = 0.0005399826445331622
gauss-legendre error = 7.011441665975837e-12 

quadrature result = 0.000539983071924356
clenshaw-curtis error = 4.3440263539085144e-10 

6 agm res = 0.0005399826375217207
error = 1.0842021724855044e-19
6 agm res = 0.0005399826375217207
error = 1.0842021724855044e-19


In [57]:
a = 0.012341234
b = 0.05123
f = get_f(a, b)
integration = quad(f, 0, np.pi/2)
print('scipy integration result =', integration[0], '\n')
gauss_legendre(15, f)
eps = clenshaw_curtis(15, f)
agm(a, b, eps)
agm(a, b, 1e-10)

scipy integration result = 55.372594344972384 

quadrature result = 55.37259488587961
gauss-legendre error = 5.409072230122547e-07 

quadrature result = 55.37262346931333
clenshaw-curtis error = 2.9124340947817018e-05 

4 agm res = 55.37275678050571
error = 0.00016243553332628835
5 agm res = 55.37259434509143
error = 1.1904432994924719e-10


In [68]:
def agm_n(a, b, n, verbose=False):
    a_prev = a
    b_prev = b
    a_next = np.sqrt(a * b) 
    b_next = (a + b) / 2
    for _ in range(n):
        a_next = np.sqrt(a_prev * b_prev)
        b_next = (a_prev + b_prev) / 2
        a_prev = a_next
        b_prev = b_next
    res = np.pi / 2 / a_next
    print(f'{n} agm res =', res)
    integration = quad(f, 0, np.pi/2)
    print('error =', np.abs(res - integration[0]), '\n')

In [69]:
a = 5
b = 10
f = get_f(a, b)
integration = quad(f, 0, np.pi/2)
print('scipy integration result =', integration[0], '\n')
for i in range(10):
    agm_n(a, b, i + 1)

scipy integration result = 0.21565156474996452 

1 agm res = 0.22214414690791828
error = 0.006492582157953758 

2 agm res = 0.21569831091347438
error = 4.674616350985872e-05 

3 agm res = 0.2156515672823984
error = 2.532433873714268e-09 

4 agm res = 0.2156515647499643
error = 2.220446049250313e-16 

5 agm res = 0.2156515647499643
error = 2.220446049250313e-16 

6 agm res = 0.2156515647499643
error = 2.220446049250313e-16 

7 agm res = 0.2156515647499643
error = 2.220446049250313e-16 

8 agm res = 0.2156515647499643
error = 2.220446049250313e-16 

9 agm res = 0.2156515647499643
error = 2.220446049250313e-16 

10 agm res = 0.2156515647499643
error = 2.220446049250313e-16 



In [70]:
a = 1234
b = 5321
f = get_f(a, b)
integration = quad(f, 0, np.pi/2)
print('scipy integration result =', integration[0], '\n')
for i in range(10):
    agm_n(a, b, i + 1)

scipy integration result = 0.0005399826375217206 

1 agm res = 0.0006130073301670527
error = 7.302469264533213e-05 

2 agm res = 0.0005420276260265462
error = 2.044988504825611e-06 

3 agm res = 0.0005399845627413885
error = 1.9252196679312578e-09 

4 agm res = 0.0005399826375234367
error = 1.7160751986100564e-15 

5 agm res = 0.0005399826375217207
error = 1.0842021724855044e-19 

6 agm res = 0.0005399826375217207
error = 1.0842021724855044e-19 

7 agm res = 0.0005399826375217207
error = 1.0842021724855044e-19 

8 agm res = 0.0005399826375217207
error = 1.0842021724855044e-19 

9 agm res = 0.0005399826375217207
error = 1.0842021724855044e-19 

10 agm res = 0.0005399826375217207
error = 1.0842021724855044e-19 



In [71]:
a = 0.0012341234
b = 0.05123
f = get_f(a, b)
integration = quad(f, 0, np.pi/2)
print('scipy integration result =', integration[0], '\n')
for i in range(10):
    agm_n(a, b, i + 1)

scipy integration result = 99.80198191072782 

1 agm res = 197.5506239027773
error = 97.74864199204947 

2 agm res = 108.76343840216411
error = 8.961456491436294 

3 agm res = 99.97896303648157
error = 0.17698112575375546 

4 agm res = 99.8020601636792
error = 7.825295138275123e-05 

5 agm res = 99.80198191074315
error = 1.5333512237702962e-11 

6 agm res = 99.80198191072782
error = 0.0 

7 agm res = 99.80198191072782
error = 0.0 

8 agm res = 99.80198191072782
error = 0.0 

9 agm res = 99.80198191072782
error = 0.0 

10 agm res = 99.80198191072782
error = 0.0 

